In [6]:

import os.path
import pandas as pn

In [7]:
import numpy as np
from itertools import count
from collections import OrderedDict

In [8]:
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torch.utils.data import DataLoader, TensorDataset, Dataset
import torch.utils.data

torch.set_default_dtype(torch.double)
torch.manual_seed(1)

In [9]:
import torch.nn.utils.rnn as rnn_utils

In [10]:
def tensor_seq_to_onehot(s,nlabels):
    bar = torch.zeros(s.shape[0],1,nlabels)
    bar.scatter_(-1,s.view(-1,1,1),1)
    return bar

In [11]:
import sys
sys.path.append("code")

In [13]:
from utils.rnns import *

In [14]:
class MyAuto(torch.nn.Module):
    def __init__(self,N_SYMBOLS):
        super(MyAuto,self).__init__()
        self.N_SYMBOLS = N_SYMBOLS
        self.N_hidden = 30
        self.D_embed = 21
        
        self.everything = RNN_Layers(
            RNN_Input_Embedding( nn.Embedding(N_SYMBOLS,self.D_embed,padding_idx=0) )
            , nn.RNN(self.D_embed,self.N_hidden,2,dropout=0.1)
            , RNN_FFLayer(nn.Linear(self.N_hidden,N_SYMBOLS))
            , RNN_FFLayer(nn.Softmax(dim=-1))
        )
    
    def forward(self, some_data):
        return self.everything(some_data)

In [15]:
model = MyAuto(20)

In [16]:
N_SYMBOLS = 20 #reserve the 1 and 0 symbol for start and stop
M_SEQS = 31
L_SEQS = (20,50)

import random as pyrand
#create sequences as 1-d long tensors
basic_sequence_lengths = torch.LongTensor(M_SEQS).random_(*L_SEQS)
basic_sequences = [torch.LongTensor(i.item()).random_(2,N_SYMBOLS) for i in basic_sequence_lengths]
for i in basic_sequences:
    i[0] = 1
    i[-1] = 0

In [17]:
#For individual sequences, it should only be two dimensions with the first dimension as time.
torch_shaped_sequences = [i.reshape(-1,1) for i in basic_sequences]

In [18]:
#Experiments about padding

In [19]:
padded_torch_seqs = rnn_utils.pad_sequence(basic_sequences)
print("padded shape, basic: " , padded_torch_seqs.shape)
print("shape after embedding: ", model.everything.layers[0](padded_torch_seqs)[0].shape)
print("Can run model on this?", model(padded_torch_seqs)[0].shape)

padded shape, basic:  torch.Size([49, 31])
shape after embedding:  torch.Size([49, 31, 21])
Can run model on this? torch.Size([49, 31, 20])


In [20]:
padded_torch_seqs = rnn_utils.pad_sequence(torch_shaped_sequences)
print("padded shape, torch_shaped: ", padded_torch_seqs.shape)
print("shape after embedding: ", model.everything.layers[0](padded_torch_seqs)[0].shape)
print("\t(Note the added extra dimension, handled in the model)")
print("Can run model on this?", model(padded_torch_seqs)[0].shape) #Because the model automagically handles that extra coordinate

padded shape, torch_shaped:  torch.Size([49, 31, 1])
shape after embedding:  torch.Size([49, 31, 21])
	(Note the added extra dimension, handled in the model)
Can run model on this? torch.Size([49, 31, 20])


In [21]:
#experiments about packing

In [24]:
packed_torch_seqs = rnn_utils.pack_sequence(basic_sequences,enforce_sorted=False)
print("packed shape :", packed_torch_seqs.data.shape)
print("shape after embedding :", model.everything.layers[0](padded_torch_seqs)[0].shape)
#try:
print("Can run model on this?",model(packed_torch_seqs)[0].data.shape)
#except Exception as e:
#    print("\n\tCan't run model. {}".format(e))

packed shape : torch.Size([1149])
shape after embedding : torch.Size([49, 31, 21])
Can run model on this? torch.Size([1149, 20])


In [25]:
packed_torch_seqs = rnn_utils.pack_sequence(torch_shaped_sequences,enforce_sorted=False)
print("packed shape :", packed_torch_seqs.data.shape)
print("shape after embedding :", model.everything.layers[0](padded_torch_seqs)[0].shape)
print("Can run model on this?",model(packed_torch_seqs)[0].data.shape)

packed shape : torch.Size([1149, 1])
shape after embedding : torch.Size([49, 31, 21])
Can run model on this? torch.Size([1149, 20])


In [26]:
#Other experiments

In [27]:
packed_torch_seqs = rnn_utils.pack_sequence(torch_shaped_sequences,enforce_sorted=False)
print("packed shape :", packed_torch_seqs.data.shape)
unpacked_torch_seqs, unpacked_lengths = rnn_utils.pad_packed_sequence(packed_torch_seqs)

print("shape after embedding :", model.everything.layers[0](padded_torch_seqs)[0].shape)
print("Can run model on this?",model(packed_torch_seqs)[0].data.shape)

packed shape : torch.Size([1149, 1])
shape after embedding : torch.Size([49, 31, 21])
Can run model on this? torch.Size([1149, 20])
